# Maps of sites with standard deviational ellipses 
Load python libraries:

In [ ]:
import numpy
import pandas
import geopandas
import pysal
import contextily
import seaborn as sns
from pointpats import centrography
from matplotlib.patches import Ellipse

Load data with mobbing information and nest locations and combine them into a dataframe:

In [ ]:
dataframe_location_2019_2020 = pd.read_csv("../../resources/original_data/FinlandNestDatafile.csv")
dataframe_location_2021 = pd.read_csv("../../resources/original_data/Finland_nestdata2021_mod.csv")
dataframe_mobbing_2019_2020 = pd.read_csv("../../resources/original_data/FinlandMobbingDatafile.csv")
dataframe_mobbing_2021 = pd.read_csv("../../resources/original_data/Finland_ExperimentData2021_mod.csv")

# combine mobbing dataframes:
dataframe_mobbing = pd.concat([dataframe_mobbing_2021,dataframe_mobbing_2019_2020], axis=0, ignore_index=True)
# combine locations:
dataframe_location_2021['Year'] = np.repeat(2021, dataframe_location_2021.shape[0])
dataframe_location = pd.concat([dataframe_location_2019_2020,dataframe_location_2021], axis=0, ignore_index=True)

dataframe_mobbing = dataframe_mobbing.drop(
    columns=['Site', 'Year', 'lat', 'long', 'Cuckoo_perch', 'New_rebuild', 'Rebuild_original']
)

data = pd.merge(dataframe_location, dataframe_mobbing, left_on='NestID', right_on='NestID', how = 'left')

Define the function to plot maps with standard deviational ellipses:

The function takes the following parameters:
- site: name of the site you want to print maps of
- db: data with mobbing and location information of the 'site'
- years: if not specified all years are printed
- plot_nests: defines whether to plot nests (dots) on the maps or not. Accepted values are $True$, $False$.
- plot_median: plot the median for all nests for each year. Accepted values are $True$, $False$.
- plot_mean: plot the mean for all nests for each year. Accepted values are $True$, $False$.
- plot_ellipses: set to $False$ if you do not want to plot standard deviational ellipses, otherwise $True$.
- map_size: define the size of the maps. The default value is 20.
- margin: define the "zoom" of the maps. The default value is 0.0015.

In [ ]:
def plot_ellipses(
    site, db,
    years = [],
    plot_nests = True, plot_median = True, plot_mean = True, plot_ellipses = True,
    map_size = 20, margin = 0.0015
):
    # filter data if we want to visualize only a subset of the dataset
    if len(years) != 0:
        db = db[[y in years for y in db.Year]].reset_index().copy()
    
    # define the map
    f, axs = plt.subplots(1, 3, figsize=(20, 8))
    
    # to set the size of the map
    lat_min = np.min(db['lat'])
    lat_max = np.max(db['lat'])
    lon_min = np.min(db['long'])
    lon_max = np.max(db['long'])
    
    # set colors
    col = sns.color_palette(n_colors=len(db.Year.unique()))
    
    # plot all birds (shy and aggressive and no-data birds):
    
    for i in range(len(db.Year.unique())):
        axs[0].set_xlim(lon_min - margin, lon_max + margin)
        axs[0].set_ylim(lat_min - margin, lat_max + margin)
        year = db.Year.unique()[i]
    
        # plot the points:
        if plot_nests:
            axs[0].scatter(db[db.Year == year]['long'], db[db.Year == year]['lat'], 
                       label = str(year) + ': nest', color = col[i])
    
        # add mean and median
        if plot_mean:
            mean_center = centrography.mean_center(db[db.Year == year][['long', 'lat']])
            axs[0].scatter(*mean_center, color=col[i], marker='x', label= 'mean center' , s = 100)
        if plot_median:
            med_center = centrography.euclidean_median(db[db.Year == year][['long', 'lat']])
            axs[0].scatter(*med_center, color=col[i], marker='*', label= 'median center')
    
        # add ellipse:
        if plot_ellipses & (db[db.Year == year].shape[0] > 3):
            major, minor, rotation = centrography.ellipse(db[db.Year == year][['long', 'lat']])
            ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                  width=major*2, # centrography.ellipse only gives half the axis
                  height=minor*2, 
                  angle = -np.degrees(rotation), # Angles for this are in degrees, not radians
                  facecolor='none', 
                  edgecolor = col[i], linestyle='--',
                  label=' std. ellipse')
            axs[0].add_patch(ellipse)
            
        axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
            ncol=3, fancybox=True, shadow=True)

        contextily.add_basemap(
            axs[0],
            crs="EPSG:4326",
            source=contextily.providers.CartoDB.PositronNoLabels,
        )
        
    
    
    # plot only shy ones:
    db_shy = db[db.Propensity == 0].reset_index().copy()
    
    
    for i in range(len(db_shy.Year.unique())):

        axs[1].set_xlim(lon_min - margin, lon_max + margin)
        axs[1].set_ylim(lat_min - margin, lat_max + margin)
        year = db_shy.Year.unique()[i]

        # plot the points:
        if plot_nests:
            axs[1].scatter(db_shy[db_shy.Year == year]['long'], db_shy[db_shy.Year == year]['lat'], 
                       label = str(year) + ': nest', color = col[i])
    
        # add mean and median
        if plot_mean:
            mean_center = centrography.mean_center(db_shy[db_shy.Year == year][['long', 'lat']])
            axs[1].scatter(*mean_center, color=col[i], marker='x', label= 'mean center' , s = 100)
        if plot_median:
            med_center = centrography.euclidean_median(db_shy[db_shy.Year == year][['long', 'lat']])
            axs[1].scatter(*med_center, color=col[i], marker='*', label= 'median center')
    
        # add ellipse:
        if plot_ellipses & (db_shy[db_shy.Year == year].shape[0] > 3):
            major, minor, rotation = centrography.ellipse(db_shy[db_shy.Year == year][['long', 'lat']])
            ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                  width=major*2, # centrography.ellipse only gives half the axis
                  height=minor*2, 
                  angle = -np.degrees(rotation), # Angles for this are in degrees, not radians
                  facecolor='none', 
                  edgecolor = col[i], linestyle='--',
                  label=' std. ellipse')
            axs[1].add_patch(ellipse)
            
        axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
            ncol=3, fancybox=True, shadow=True)

        contextily.add_basemap(
            axs[1],
            crs="EPSG:4326",
            source=contextily.providers.CartoDB.PositronNoLabels,
        )
    
    
    # plot aggressive ones:
    
    db_agg = db[db.Propensity == 1].reset_index().copy()
    
    for i in range(len(db_agg.Year.unique())):
        axs[2].set_xlim(lon_min - margin, lon_max + margin)
        axs[2].set_ylim(lat_min - margin, lat_max + margin)
        year = db_agg.Year.unique()[i]
    
        # plot the points:
        if plot_nests:
            axs[2].scatter(db_agg[db_agg.Year == year]['long'], db_agg[db_agg.Year == year]['lat'], 
                       label = str(year) + ': nest', color = col[i])
    
        # add mean and median
        if plot_mean:
            mean_center = centrography.mean_center(db_agg[db_agg.Year == year][['long', 'lat']])
            axs[2].scatter(*mean_center, color=col[i], marker='x', label= 'mean center' , s = 100)
        if plot_median:
            med_center = centrography.euclidean_median(db_agg[db_agg.Year == year][['long', 'lat']])
            axs[2].scatter(*med_center, color=col[i], marker='*', label= 'median center')
    
        # add ellipse:
        if plot_ellipses & (db_agg[db_agg.Year == year].shape[0] > 3):
            major, minor, rotation = centrography.ellipse(db_agg[db_agg.Year == year][['long', 'lat']])
            ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                  width=major*2, # centrography.ellipse only gives half the axis
                  height=minor*2, 
                  angle = -np.degrees(rotation), # Angles for this are in degrees, not radians
                  facecolor='none', 
                  edgecolor = col[i], linestyle='--',
                  label=' std. ellipse')
            axs[2].add_patch(ellipse)
            
        axs[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
            ncol=3, fancybox=True, shadow=True)

        contextily.add_basemap(
            axs[2],
            crs="EPSG:4326",
            source=contextily.providers.CartoDB.PositronNoLabels,
        )
        
    f.suptitle(site+ ': Standard Deviational Ellipses', fontsize=16)
    axs[0].set_title('All nests')
    axs[1].set_title('Only shy')
    axs[2].set_title('Only aggressive')
    
    f.patch.set_facecolor('white')
    # comment out the following row to save plots as png images:
    #plt.savefig('sd_ellipses/sd_ellipses_'+site+'.png')

Example:

In [ ]:
site = 'Otaniemi'

db = data[data.Site == site].reset_index().copy()
plot_ellipses(site, db)